In [1]:
from allthethings import PyNetwork
from allthethings import PyBC_opt_dh
import numpy as np

In [2]:
fi = "indata/3pipes1.inp"     #location of .inp file
fc = "indata/3pipes1.config"  #location of .config file
mtype = 1                     #model used along network edges. 1 for Preissman Slot. 0 for uniform
n1 = PyNetwork(fi, fc, mtype) #a network object
print n1 
#n1.showLayout()               #print the layout of the network


Network at address 0x10220d130 with 4 nodes and 3 edges



In [3]:
#n1.showCurrentData()    
dt = n1.T/n1.M
#n1.runForwardProblem(dt)  #solve up to time T

In [4]:
#n1.showCurrentData()

In [6]:
ndof = 16
wn = np.array([1,2])
x0 = np.ones(ndof*len(wn))
opt1 = PyBC_opt_dh(fi, fc, ndof, x0, 0, wn)
opt1.solve()


[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
T = 0.500000
[1 2]
[1, 2]
